In [1]:
from utils.experimentor import *
from utils.constants import *

In [2]:
config = {
    "dataset_name": Dataset.OGBN_PROTEINS,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 10000,
    'num_of_runs': 3,
    'patience_period': 50,
    
    'batch_size': 256,
    'test_batch_size': 256,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 5,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.01,
    'num_of_layers': 2, 
    'num_heads': 1,
    'hidden_size': 64,
    'dropout': 0.5,  
    "use_layer_norm": False,
    
    'nbor_degree': 1,
    'adj_mode': AdjacencyMode.OneStep,
    'sparse': False
}

In [3]:
experimentor = Experimentor(config, logging)
experimentor.run()

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')
ModuleList(
  (0): GATConv(8, 64, heads=1)
  (1): GATConv(64, 112, heads=1)
)
Run 01:


Epoch 01:   0%|          | 0/86619 [00:00<?, ?it/s]

torch.Size([256, 112]) torch.Size([256, 112])


RuntimeError: The size of tensor a (256) must match the size of tensor b (112) at non-singleton dimension 1